In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
import folium
import sklearn
import seaborn as sns
from IPython.core.display import display, HTML
import ipywidgets as widgets
from IPython.display import clear_output
import itertools

In [2]:
def percent_to_int(df_in):
    for col in df_in.columns.values:
        if col.startswith("Percent") or col.endswith("%") or col.endswith("Rate"):
            df_in[col] = df_in[col].astype(np.object).str.replace('%', '').astype(float)
    return df_in

In [3]:
df_schools_raw = pd.read_csv('../../../input/2016 School Explorer.csv')
df_schools_raw = df_schools_raw[df_schools_raw['Grade High'] != '0K']
df_schools_raw = percent_to_int(df_schools_raw)
df_schools_raw['School Income Estimate'] = df_schools_raw['School Income Estimate'].astype(np.object).str.replace('$', '').str.replace(',', '').str.replace('.', '').astype(float)

In [4]:
print df_schools_raw.shape

(1270, 161)


In [5]:
df_schools_minor = df_schools_raw[df_schools_raw['Percent Black / Hispanic'].astype(int)>70]
print(df_schools_minor.shape)

(829, 161)


In [6]:
def grade_minority_percent_4s (df, grade, subject, thres_total_4s, thres_minority):
    out_field = ('Grade %d %s Minority 4s')%(grade, subject)
    num1 = ('Grade %d %s 4s - Black or African American')%(grade, subject)
    num2 = ('Grade %d %s 4s - Hispanic or Latino')%(grade, subject)
    den = ('Grade %d %s 4s - All Students')%(grade, subject)

    df = df[df[den].astype(int)>thres_total_4s]
    df[out_field] = (df[num1] + df[num2])/(df[den])
    grade_minority_df = df[df[out_field]>thres_minority]

    df_html = grade_minority_df[['Location Code','School Name', out_field, num1, num2, den]].head().to_html()
    display(HTML(df_html))

    return df

In [7]:
def compute_4s_minority_states():
    grade = int(toggle.value.split(" ")[1])
    subjects = ['ELA', 'Math']

    plt.figure(figsize=[19,10])
    df_all_schools = pd.DataFrame(columns=df_schools_minor.columns)
    column_list = []

    for subject in subjects:
        df_all_schools = grade_minority_percent_4s(df_schools_minor, grade, subject, thres_total_4s.value, thres_minority.value)
        out_field = ('Grade %d %s Minority 4s')%(grade, subject)
        num1 = ('Grade %d %s 4s - Black or African American')%(grade, subject)
        num2 = ('Grade %d %s 4s - Hispanic or Latino')%(grade, subject)
        den = ('Grade %d %s 4s - All Students')%(grade, subject)

In [8]:
style = {'description_width': 'initial'}

def on_value_change(change):
    clear_output()
    display(thres_total_4s)
    display(thres_minority)
    display(toggle)
    compute_4s_minority_states()
    
thres_total_4s = widgets.IntSlider(
    value = 5,
    min=0,
    max=20,
    step=1,
    description='Total Students with 4s',
    style=style,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
thres_total_4s.observe(on_value_change, names='value')
# display(thres_total_4s)

thres_minority = widgets.FloatSlider(
    value=0.7,
    min=0,
    max=1.0,
    step=0.05,
    description='Ratio of minority students',
    style=style,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
thres_minority.observe(on_value_change, names='value')
# display(thres_minority)

In [12]:
toggle = widgets.ToggleButtons(
    options=['Grade 5', 'Grade 6', 'Grade 7', 'Grade 8'],
    value='Grade 8',
    description='Grades:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Info. about Grade 5', 'Info. about Grade 6', 'Info. about Grade 7', 'Info. about Grade 8'],
#     icons=['check'] * 3
)

In [13]:
display(toggle)
display(thres_total_4s)
display(thres_minority)

compute_4s_minority_states()

def on_click(change):
    clear_output()
    
    display(thres_total_4s)
    display(thres_minority)
    display(toggle)
    
    compute_4s_minority_states()
    
toggle.observe(on_click, 'value')

IntSlider(value=16, continuous_update=False, description=u'Total Students with 4s', max=20, style=SliderStyle(description_width=u'initial'))

FloatSlider(value=0.25, continuous_update=False, description=u'Ratio of minority students', max=1.0, step=0.05, style=SliderStyle(description_width=u'initial'))

ToggleButtons(button_style=u'info', description=u'Grades:', options=('Grade 5', 'Grade 6', 'Grade 7', 'Grade 8'), tooltips=[u'Info. about Grade 5', u'Info. about Grade 6', u'Info. about Grade 7', u'Info. about Grade 8'], value='Grade 5')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Location Code,School Name,Grade 5 ELA Minority 4s,Grade 5 ELA 4s - Black or African American,Grade 5 ELA 4s - Hispanic or Latino,Grade 5 ELA 4s - All Students
263,09X053,P.S. 053 BASHEER QUISIM,0.958333,11,12,24
311,10X007,MILTON FEIN SCHOOL,0.740741,7,13,27
330,10X086,P.S. 086 KINGSBRIDGE HEIGHTS,0.909091,3,17,22
378,11X096,P.S. 096 RICHARD RODGERS,0.722222,5,8,18
389,11X153,P.S. 153 HELEN KELLER,0.600000,10,2,20


,Location Code,School Name,Grade 5 Math Minority 4s,Grade 5 Math 4s - Black or African American,Grade 5 Math 4s - Hispanic or Latino,Grade 5 Math 4s - All Students
113,04M083,P.S. 083 LUIS MUNOZ RIVERA,0.705882,1,11,17
221,08X036,P.S. 036 UNIONPORT,0.600000,1,11,20
311,10X007,MILTON FEIN SCHOOL,0.575758,6,13,33
325,10X056,P.S. 056 NORWOOD HEIGHTS,0.529412,1,8,17
330,10X086,P.S. 086 KINGSBRIDGE HEIGHTS,0.809524,2,32,42
